# Tutorial su Phi-4 Mini ONNX per Chiamate di Funzioni Parallele

Questo notebook dimostra come utilizzare Phi-4 Mini con ONNX Runtime GenAI per effettuare chiamate di funzioni parallele. Le chiamate di funzioni permettono al modello di invocare in modo intelligente strumenti e API esterni basandosi sulle richieste dell'utente.

## Panoramica

In questo tutorial imparerai a:
- Configurare Phi-4 Mini con ONNX Runtime GenAI
- Definire schemi di funzioni per prenotare voli e hotel
- Utilizzare la generazione guidata con la grammatica Lark per ottenere output strutturati
- Eseguire chiamate di funzioni parallele per scenari complessi di prenotazione viaggi

## Prerequisiti

Prima di eseguire questo notebook, assicurati di:
- Aver scaricato il modello Phi-4 Mini ONNX
- Aver installato il pacchetto `onnxruntime-genai`
- Avere una conoscenza di base dei concetti relativi alle chiamate di funzioni


## Passaggio 1: Importa le librerie necessarie

Per prima cosa, importeremo le librerie necessarie per l'implementazione della chiamata alla funzione.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Passaggio 2: Configurazione e impostazione del modello

Ora configureremo il modello Phi-4 Mini ONNX. Assicurati di sostituire il percorso con la directory effettiva del tuo modello.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Passaggio 3: Configurare i Parametri di Generazione

Configura i parametri di generazione per controllare il comportamento dell'output del modello. Queste impostazioni garantiscono risposte deterministiche e mirate per la chiamata delle funzioni.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Passaggio 4: Definire le Funzioni Disponibili

Qui definiamo le funzioni che il nostro assistente AI può utilizzare. In questo esempio, abbiamo due funzioni relative ai viaggi:
1. **booking_flight_tickets**: Per prenotare voli tra aeroporti
2. **booking_hotels**: Per prenotare sistemazioni in hotel

Le definizioni delle funzioni seguono il formato dello schema di chiamata delle funzioni di OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Passaggio 5: Funzioni di supporto per la generazione della grammatica

Queste funzioni di supporto convertono le definizioni delle nostre funzioni nel formato di grammatica Lark, utilizzato da ONNX Runtime GenAI per la generazione guidata. Questo garantisce che il modello produca chiamate di funzione valide nel formato JSON corretto.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Passaggio 6: Testare la Generazione della Grammatica

Testiamo le nostre funzioni di generazione della grammatica per vedere come convertono le definizioni degli strumenti nel formato corretto.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Passaggio 7: Preparare il Prompt di Sistema e il Generatore

Ora creeremo il prompt di sistema che informa il modello sugli strumenti disponibili e configureremo il generatore con parametri di generazione guidata.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Passaggio 8: Inizializzare il Generatore con Generazione Guidata

Ora creeremo il generatore con i parametri configurati e applicheremo la grammatica Lark per la generazione guidata.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Passaggio 9: Testare la Chiamata di Funzioni in Parallelo

Ora testiamo la nostra configurazione con uno scenario complesso di prenotazione di viaggi che richiede la chiamata di più funzioni.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generare Chiamate di Funzione

Il modello ora genererà chiamate di funzione strutturate basate sulla richiesta dell'utente. Grazie alla generazione guidata, l'output sarà in formato JSON valido che può essere eseguito direttamente.

**Output Atteso**: Il modello dovrebbe generare chiamate di funzione per:
1. `booking_flight_tickets` con i dettagli da Pechino (PEK) a Parigi (CDG)
2. `booking_hotels` con i dettagli dell'alloggio a Parigi

Esegui la cella qui sotto per vedere la generazione in tempo reale:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Conclusione

🎉 **Congratulazioni!** Hai implementato con successo le chiamate di funzione parallele con Phi-4 Mini utilizzando ONNX Runtime GenAI.

### Cosa Hai Imparato:

1. **Configurazione del Modello**: Come configurare Phi-4 Mini con ONNX Runtime GenAI
2. **Definizione delle Funzioni**: Come definire gli schemi delle funzioni per le chiamate AI
3. **Generazione Guidata**: Come utilizzare la grammatica Lark per generare output strutturati
4. **Chiamate di Funzioni Parallele**: Come gestire richieste complesse che richiedono più chiamate di funzione

### Vantaggi Principali:

- ✅ **Output Strutturato**: La generazione guidata garantisce chiamate di funzione JSON valide
- ✅ **Elaborazione Parallela**: Gestisci più chiamate di funzione in una singola richiesta
- ✅ **Prestazioni Elevate**: ONNX Runtime offre inferenze ottimizzate
- ✅ **Schema Flessibile**: Facile aggiungere o modificare le definizioni delle funzioni

### Risorse:

- [Documentazione di Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Documentazione di ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Best Practices per le Chiamate di Funzione](https://platform.openai.com/docs/guides/function-calling)



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
